In [1]:
import pandas as pd

# Load each sheet explicitly
od_df = pd.read_excel('../data/DataSet_2_OD.xlsx', sheet_name='OD_Matrix16_238')
link_df = pd.read_excel('../data/DataSet_2_Link parameter.xlsx', sheet_name='Link parameters_16')
yard_df = pd.read_excel('../data/DataSet_2_Yard Parameter.xlsx', sheet_name='Yard Parameter_16')

# Display top few rows of each
print("OD Data:")
display(od_df.head())

print("\nLink Data:")
display(link_df.head())

print("\nYard Data:")
display(yard_df.head())


OD Data:


,YARD,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16
0,Y01,0,142,27,36,191,4,102,153,92,182,21,116,118,95,47,126
1,Y02,171,0,138,69,112,67,99,35,94,103,11,122,133,73,64,141
2,Y03,111,53,0,68,147,44,62,157,37,59,123,141,129,178,116,35
3,Y04,190,42,88,0,106,40,142,64,48,46,5,90,129,170,150,6
4,Y05,101,193,148,29,0,23,84,154,156,40,166,176,131,108,144,39



Link Data:


,No.,NodeA,NodeB,CapaAB,Length
0,1,Y01,Y02,40,200
1,2,Y02,Y03,30,183
2,3,Y03,Y04,16,225
3,4,Y05,Y06,18,221
4,5,Y06,Y07,35,201



Yard Data:


,No.,Yard,RC,ST,COST,AP
0,1,Y01,422,14,4.20,10.8
1,2,Y02,129,18,3.92,10.5
2,3,Y03,511,16,4.04,11.3
3,4,Y04,502,15,3.95,11.2
4,5,Y05,204,15,4.13,11.1


In [2]:
import networkx as nx

# ---- Parse OD Demand into a list of (origin, dest, demand) ----
od_long = od_df.set_index('YARD').stack().reset_index()
od_long.columns = ['origin', 'destination', 'demand']
od_long = od_long[od_long['demand'] > 0]  # remove 0 demand

# ---- Create Graph from Link Data ----
G = nx.DiGraph()

for _, row in link_df.iterrows():
    G.add_edge(row['NodeA'], row['NodeB'], 
               capacity=row['CapaAB'], 
               cost=row['Length'])  # or use distance/yard cost if needed

# ---- Create Yard Info Dictionary ----
yard_info = yard_df.set_index('Yard')[['RC', 'ST', 'COST', 'AP']].to_dict('index')

# ---- Check result ----
print(f"✅ Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
print(f"✅ Total OD pairs: {len(od_long)}")
print(f"✅ Total yards: {len(yard_info)}")


✅ Graph has 16 nodes and 48 edges
✅ Total OD pairs: 238
✅ Total yards: 16


In [12]:
import sys
sys.path.append('../models')

from tspb_model import build_tspb_model

# Sample one OD pair with high demand
od_sample = od_long.head(5)

print("Sample OD pair(s):", od_sample)
print("Graph sample edges:", list(G.edges(data=True))[:5])
print("Yard info sample:", list(yard_info.items())[:3])

# Build and solve the model
model, x_vars = build_tspb_model(od_sample, G, yard_info)

model.optimize()


Sample OD pair(s):   origin destination  demand
1    Y01         Y02     142
2    Y01         Y03      27
3    Y01         Y04      36
4    Y01         Y05     191
5    Y01         Y06       4
Graph sample edges: [('Y01', 'Y02', {'capacity': 200, 'cost': 200}), ('Y01', 'Y05', {'capacity': 200, 'cost': 173}), ('Y02', 'Y03', {'capacity': 200, 'cost': 183}), ('Y02', 'Y06', {'capacity': 200, 'cost': 178}), ('Y02', 'Y01', {'capacity': 200, 'cost': 200})]
Yard info sample: [('Y01', {'RC': 422, 'ST': 14, 'COST': 4.2, 'AP': 10.8}), ('Y02', {'RC': 129, 'ST': 18, 'COST': 3.92, 'AP': 10.5}), ('Y03', {'RC': 511, 'ST': 16, 'COST': 4.04, 'AP': 11.3})]
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13420H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 128 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x19d473b8
Variable types: 0 co